<h1> Análise Exploratória <h1>

In [3]:
import pandas as pd
from statsbombpy import sb
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import ConvexHull
from scipy.interpolate import make_interp_spline
from matplotlib.colors import to_rgba, ListedColormap
import matplotlib.patches as patches

In [4]:
MATCH_ID = 3942819
match_events_df = sb.events(match_id= MATCH_ID)
match_360_df = pd.read_json(f"../Data/{MATCH_ID}.json")
df = pd.merge(left=match_events_df, right=match_360_df, left_on='id', right_on='event_uuid', how='left')

c:\Users\arthu\anaconda3\envs\football\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3485 entries, 0 to 3484
Data columns (total 92 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   50_50                        2 non-null      object 
 1   bad_behaviour_card           3 non-null      object 
 2   ball_receipt_outcome         66 non-null     object 
 3   block_deflection             1 non-null      object 
 4   block_save_block             1 non-null      object 
 5   carry_end_location           910 non-null    object 
 6   clearance_aerial_won         3 non-null      object 
 7   clearance_body_part          12 non-null     object 
 8   clearance_head               7 non-null      object 
 9   clearance_left_foot          1 non-null      object 
 10  clearance_right_foot         4 non-null      object 
 11  counterpress                 68 non-null     object 
 12  dribble_outcome              12 non-null     object 
 13  duel_outcome      

In [6]:
df.head()

,50_50,bad_behaviour_card,ball_receipt_outcome,block_deflection,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure,event_uuid,visible_area,freeze_frame
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Netherlands,941,00:00:00.000,Starting XI,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"{'formation': 3421, 'lineup': [{'player': {'id...",England,768,00:00:00.000,Starting XI,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Netherlands,941,00:00:00.000,Half Start,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,England,768,00:00:00.000,Half Start,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Netherlands,941,00:00:00.000,Half Start,NaN,NaN,NaN,NaN


In [7]:
df['type'].value_counts()

type
Pass                 1065
Ball Receipt*        1020
Carry                 910
Pressure              173
Ball Recovery          55
Duel                   46
Block                  27
Foul Won               23
Foul Committed         23
Goal Keeper            19
Miscontrol             16
Interception           16
Shot                   16
Dispossessed           14
Dribble                12
Clearance              12
Substitution            9
Injury Stoppage         5
Half Start              4
Half End                4
Bad Behaviour           3
Dribbled Past           3
Referee Ball-Drop       2
50/50                   2
Tactical Shift          2
Starting XI             2
Player On               1
Player Off              1
Name: count, dtype: int64

data frame in which freeze_frame has an attribute

In [13]:
df_on_360 = df[df['freeze_frame'].notna()]
df_on_360.head()

,50_50,bad_behaviour_card,ball_receipt_outcome,block_deflection,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure,event_uuid,visible_area,freeze_frame
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,England,768,00:00:00.355,Pass,NaN,25dfc952-7a85-464f-b884-982064d46cc9,"[82.05880352279489, 80.0, 37.469993200629204, ...","[{'teammate': True, 'actor': False, 'keeper': ..."
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,England,768,00:00:05.294,Pass,True,8a920161-a913-4ce5-be61-9d6ae7466469,"[40.07701261372162, 75.76529270581614, 10.9116...","[{'teammate': True, 'actor': True, 'keeper': T..."
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Netherlands,941,00:00:08.732,Pass,NaN,c6e3a4a2-d078-4c2d-b94d-ff721bcdf50b,"[0.0, 80.0, 0.0, 42.04634322968843, 27.5085434...","[{'teammate': True, 'actor': False, 'keeper': ..."
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,England,768,00:00:11.552,Pass,NaN,a53891bd-f254-4786-adee-ec2ecd78e857,"[84.87329142496212, 77.05638219685684, 50.6812...","[{'teammate': True, 'actor': False, 'keeper': ..."
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Netherlands,941,00:00:23.802,Pass,NaN,a5bb6f4b-aad7-48dc-a1ee-95bf94f439e3,"[12.459729753971867, 80.0, 40.24112294288676, ...","[{'teammate': True, 'actor': False, 'keeper': ..."


Accessing for reference

In [66]:
df_on_360['freeze_frame'].iloc[0][0]  # First dictionary in the list


{'teammate': True,
 'actor': False,
 'keeper': True,
 'location': [22.697541663925428, 41.01277368603656]}

- Each row in df_freeze corresponds to a row in df_on_360.
- Each column in df_freeze corresponds to an element from the list inside freeze_frame for that row.
- Each cell contains a dictionary with details about a player at the time of the shot.

In [ ]:
flat_list = [item for sublist in df_on_360['freeze_frame'] for item in sublist]
df_freeze = df_on_360['freeze_frame'].apply(pd.Series)

In [78]:
print(df_freeze.shape)
df_freeze.head()

(2951, 22)


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
6,"{'teammate': True, 'actor': False, 'keeper': T...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...",...,"{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...",NaN
7,"{'teammate': True, 'actor': True, 'keeper': Tr...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...",...,"{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...",NaN,NaN,NaN
8,"{'teammate': True, 'actor': False, 'keeper': T...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': True, 'keeper': Fa...","{'teammate': False, 'actor': False, 'keeper': ...",...,"{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...",NaN,NaN,NaN,NaN
9,"{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': True, 'keeper': Fa...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...",...,"{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...",NaN,NaN
10,"{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': True, 'actor': False, 'keeper': F...","{'teammate': False, 'actor': False, 'keeper': ...","{'teammate': True, 'actor': T

Checking again for reference

In [77]:
df_freeze.iloc[0][0]

{'teammate': True,
 'actor': False,
 'keeper': True,
 'location': [22.697541663925428, 41.01277368603656]}